In [1]:
!pip install python-igraph
!pip install cairocffi

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

     |████████████████████████████████| 3.1 MB 13.8 MB/s 
     |████████████████████████████████| 88 kB 5.8 MB/s 
  Created wheel for cairocffi: filename=cairocffi-1.3.0-py3-none-any.whl size=89668 sha256=2816aa59da9ad8dd0ee75582107acbc5b41f29a12641e8a72f6a26053b354bcf
  Stored in directory: /root/.cache/pip/wheels/4e/ca/e1/5c8a9692a27f639a07c949044bec943f26c81cd53d3805319f
Successfully built cairocffi
     |████████████████████████████████| 7.9 MB 14.0 MB/s 
     |████████████████████████████████| 3.5 MB 14.3 MB/s 
     |████████████████████████████████| 145 kB 13.7 MB/s 
     |████████████████████████████████| 112 kB 66.3 MB/s 
     |████████████████████████████████| 74 kB 2.2 MB/s 
     |████████████████████████████████| 596 kB 88.6 MB/s 


In [2]:
!pip install pysam

     |████████████████████████████████| 14.9 MB 9.9 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
gnndatasetpath = "/content/gdrive/Shareddrives/FYP/DataSets/Gnn-Dataset/Test_Datasets"

In [ ]:
_DEBUG = True

In [ ]:
import re
from igraph import *
from collections import defaultdict
import os
import random

%matplotlib inline
import torch
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

from pysam import FastaFile


In [ ]:
## Functions

In [ ]:
def visualize_graph(G, color):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    nx.draw_networkx(G, pos=nx.spring_layout(G, seed=42), with_labels=False,
                     node_color=color, cmap="Set2")
    plt.show()


def visualize_embedding(h, color, epoch=None, loss=None):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    h = h.detach().cpu().numpy()
    plt.scatter(h[:, 0], h[:, 1], s=140, c=color, cmap="Set2")
    if epoch is not None and loss is not None:
        plt.xlabel(f'Epoch: {epoch}, Loss: {loss.item():.4f}', fontsize=16)
    plt.show()

In [ ]:
# TODO: Update the function with the library
def predict(sequence):
  return [random.uniform(0, 1), random.uniform(0, 1)]

In [ ]:
# TODO: Update the function with the library
def get_label_sequences(fastafile):
  start = 'NODE_'
  end = '_length_'
  features = {}
  
  fasta_object = FastaFile(fastafile)
  fasta_references = fasta_object.references

  for name in fasta_references:
    attributes = {}

    contig_num = int(str(int(re.search('%s(.*)%s' % (start, end), name).group(1))-1))
    predicted_values = predict(fasta_object.fetch(reference = name))

    attributes["features"] = predicted_values
    attributes["class"] = 0.5
    attributes["train_sem"] = False

    # TODO Change this
    color_value = sum(predicted_values) / len(predicted_values)

    if color_value < 0.4:
      attributes["class"] = 0
      attributes["train_sem"] = True
    elif color_value > 0.8:
      attributes["class"] = 1
      attributes["train_sem"] = True

    features[contig_num] = attributes
  return features

In [ ]:

def get_the_segment_contig_map(contigfilepath):
  paths = {}
  segment_contigs = {}
  node_count = 0

  with open(contigfilepath) as file:
    name = file.readline()
    path = file.readline()
    
    while name != "" and path != "":
            
        while ";" in path:
            path = path[:-2]+","+file.readline()
        
        start = 'NODE_'
        end = '_length_'
        contig_num = int(str(int(re.search('%s(.*)%s' % (start, end), name).group(1))-1))
        
        segments = path.rstrip().split(",")
        
        if contig_num not in paths:
            node_count += 1
            paths[contig_num] = [segments[0], segments[-1]]
        
        for segment in segments:
            # TODO Check this
            if segment.endswith("+") or segment.endswith("-"):
                segment = segment[:-1]
            if segment not in segment_contigs:
                segment_contigs[segment] = set([contig_num])
            else:
                segment_contigs[segment].add(contig_num)
        
        name = file.readline()
        path = file.readline()

    return segment_contigs, paths, node_count

In [ ]:
def adjust_weights(weight_list):
  summation = 0
  count = 0
  none_indexes = []
  for i in range(len(weight_list)):
    if weight_list[i] == None:
      none_indexes.append(i)
      continue
    count += 1
    summation += weight_list[i]
  
  average_weight = summation / count
  for i in none_indexes:
    weight_list[i] = average_weight

  return weight_list

In [ ]:
#make segment_contigs global
def generate_edge_tensor(gfafilepath, segment_contigs):
  source_list = []
  destination_list= []
  weight_list = []
  isNeedToAdjustWeights = False

  with open(gfafilepath) as file:
    line = file.readline()
    while line != "":
      # Identify lines with link information
      if "L" in line:
          strings = line.split("\t")
          seg1, seg2 = strings[1], strings[3]
          weight = strings[5].strip()
          contig1 = segment_contigs[seg1]
          contig2 = segment_contigs[seg2]
          for cont1 in contig1:
            for cont2 in contig2:
              source_list.append(cont1)
              destination_list.append(cont2)
              if weight.isnumeric():
                weight_list.append(int(weight))
              elif weight[:-1].isnumeric():
                weight_list.append(int(weight[:-1]))
              else:
                weight_list.append(None)
                isNeedToAdjustWeights = True
      line = file.readline()
    if isNeedToAdjustWeights:
      weight_list = adjust_weights(weight_list)
    return source_list, destination_list, weight_list


In [ ]:
def generate_tensors(features, source_list, destination_list, weight_list, node_count):
  feature_list = []
  catergory_list = []
  train_data = []
  for i in range(node_count):
    node_features_dict = features[i]
    catergory_list.append(node_features_dict["class"])
    train_data.append(node_features_dict["train_sem"])
    feature_list.append(node_features_dict["features"])
  
  node_features = torch.DoubleTensor(feature_list).unsqueeze(1)
  y = torch.LongTensor(catergory_list)
  print(source_list)
  print(destination_list)
  print(feature_list)
  edge_index = torch.tensor([source_list, destination_list], dtype=torch.long)
  edge_attr = torch.DoubleTensor(weight_list)

  data = Data(x = node_features, edge_index = edge_index, y = y, edge_attr = edge_attr)
  return data

In [ ]:
def generate_data(gfafilepath, contigfilepath, fastafilepath):
  features = get_label_sequences(fastafilepath)
  segment_contigs, paths, node_count = get_the_segment_contig_map(contigfilepath)
  source_list, destination_list, weight_list = generate_edge_tensor(gfafilepath, segment_contigs)
  data = generate_tensors(features, source_list, destination_list, weight_list, node_count)
  return data

In [ ]:
def process():
  directory = gnndatasetpath
  data = None
  for datafolder in os.listdir(directory):
      dataset = os.path.join(directory, datafolder)
      gfafilepath = str(dataset) + "/assembly_graph_with_scaffolds.gfa"
      contigfilepath = str(dataset) + "/contigs.paths"
      fastafilepath = str(dataset) + "/contigs.fasta"

      data = generate_data(gfafilepath, contigfilepath, fastafilepath)

      G = to_networkx(data, to_undirected=True)
      visualize_graph(G, color=data.y)

      if _DEBUG:
        return data
        break

In [ ]:
data = process()

In [ ]:
### Graph Neural Network Task

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, inputfeatures = 2, hidden_channels = 2, num_classes = 2):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(inputfeatures, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [ ]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x, data.edge_index)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test():
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc

def iterate():
  for epoch in range(1, 101):
      loss = train()
      print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

In [ ]:
model = GCN(inputfeatures = 2, hidden_channels = 2, num_classes = 2)
print(model)

In [ ]:
model.eval()

In [ ]:
data.x
# data.edge_index

In [ ]:
out = model(data.x, data.edge_index)
# visualize(out, color=data.y)

In [ ]:
iterate()


In [ ]:
# # TODO: Update the function with the library
# def get_label_sequences(fastafile):
#   start = 'NODE_'
#   end = '_length_'
#   features = {}
  
#   with open(fastafile, 'r') as file:
#     line = file.readline()
#     while line != "":
#       attributes = {}
#       if line.startswith(">"):

#         name = line.lstrip(">")
#         contig_num = str(int(re.search('%s(.*)%s' % (start, end), name).group(1))-1)
#         predicted_values = predict("")

#         attributes["features"] = predicted_values
#         attributes["class"] = 0.5
#         attributes["train_sem"] = False

#         # TODO Change this
#         color_value = sum(predicted_values) / len(predicted_values)

#         if color_value < 0.4:
#           attributes["class"] = 0
#           attributes["train_sem"] = True
#         elif color_value > 0.8:
#           attributes["class"] = 1
#           attributes["train_sem"] = True

#         features[contig_num] = attributes
#         line = file.readline()

#     return features